In [ ]:
import pandas as pd

In [ ]:
%%bash
git clone https://github.com/Jaehun-Kim22/SCIC_Project.git

In [ ]:
## loads excel
xls = pd.ExcelFile('SCIC_Project/assets/data.xlsx')
training_data = pd.read_excel(xls, '학습데이터')
priority = pd.read_excel(xls, '우선순위')
data_without_ground_truth = training_data.values[:, 2]
!pip install konlpy

In [ ]:
## Kkma sentence
from konlpy.tag import Kkma
import copy
import time
kkma = Kkma()


def perform_Kkma(text):
  temp_kkma = kkma.sentences(text)
  if [text] != temp_kkma:
    new_string_to_add = ''
    new_string_to_add += ' '.join(temp_kkma)
    return new_string_to_add
  else:
    return text


s = time.time()

training_data_copy_kkma = copy.deepcopy(training_data)

cnt = 0
column_added_kkma = []
for text in training_data['발화']:
  column_added_kkma.append(perform_Kkma(text))
  cnt += 1
training_data_copy_kkma['Kkma'] = column_added_kkma

print(training_data.values[:, 2][:20])
print(training_data_copy_kkma.values[:, 10][:20])
e = time.time()

print(e - s)